In [65]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import stats

# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import os
import pickle
import time
from IPython.display import clear_output

In [68]:
dir_inter = r'Z:\All_Staff\home\lan\Data\2P_images\mat_inter/'.replace('\\', '/')
dir_file = dir_inter + 'adp_dataset_master.xlsx'
data_info = pd.read_excel(dir_file)
data_info.head()

meta = data_info[(data_info.paradigm == 'grating') & ((data_info.area == 'LM') | (data_info.area == 'LI'))]
meta.area.value_counts()

nset = meta.shape[0]
for i in np.arange(nset):
    # clear_output(wait=True)
    print('set ', i+1, 'of', nset)
    mouse = meta.iloc[i].mouse.astype(str)
    date = meta.iloc[i].date.astype(str)
    area = meta.iloc[i].area
    irun = '00' + meta.iloc[i].num.astype(int).astype(str)
    print(mouse, date, irun, area)

    ## check if resp_base_trialwise exists
    ## load data
    while True:
        try:
            try:
                dir_sub = area + '_i' + mouse + '_' + date + '_cellpose'
                dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
            except:
                dir_sub = area + '_i' + mouse + '_' + date + ''
                dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
            break
        except:
            print('set ', i+1, 'of', nset)
            print(mouse, date, irun)
            print('waiting for tif')
            time.sleep(60)
            clear_output(wait=True)
            continue

    if len(dfof_trialwise['dfof_ad_trial'].shape) == 2: # if contains only one ISI, likely 250
        mode = 'only one isi'
        print(f'mode is {mode} (only one isi, grat_SF or bunny)')
        print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0].shape)
        dfof_ad_trial = dfof_trialwise['dfof_ad_trial'] # do not subtract baseline here: stim resp will be compared to base resp
        dfof_base_trial = dfof_trialwise['dfof_base_trial']

    if len(dfof_trialwise['dfof_ad_trial'].shape) == 3: # if contains multiple ISI, likely inf-750-250
        mode = 'grat_8ori_3isi'
        print(f'mode is {mode} (grating with 8 orientations and 3 ISI)')
        print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0,0].shape)
        dfof_tg_trial = dfof_trialwise['dfof_tg_trial'][:,:,0] # only use the first ISI, aka no adapter trials
        dfof_base2_trial = dfof_trialwise['dfof_base2_trial'][:,:,0]
        print('to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. \n 8 possible target orientations will cover all needed cells')

    ## stats
    if mode == 'only one isi':
        print(mode)

        ncell = dfof_ad_trial.shape[0]
        nstim = dfof_ad_trial.shape[1]

        ## according to Ohki 2020
        p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
        p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
        p_ttest = np.ones((ncell, nstim)) * np.nan
        evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

        for icell in tqdm(np.arange(ncell)):
            base_cell_anova = np.concatenate([dfof_base_trial[icell, stim] for stim in range(nstim)])
            stim_cell_anova = [] 
            for istim in np.arange(nstim):
                stim_cell_anova.append(np.array(dfof_ad_trial[icell, istim]).flatten())

                base_cell = dfof_base_trial[icell, istim]
                stim_cell = dfof_ad_trial[icell, istim]
                _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
                p_ttest[icell, istim] = p_ttest_i

                evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
                evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
                evoked[icell, istim] = evoked_i
            
            _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            p_anova[icell] = p_anova_cell
            _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            p_kruskal[icell] = p_kruskal_cell

    elif mode == 'grat_8ori_3isi':
        print(mode)

        ncell = dfof_tg_trial.shape[0]
        nstim = dfof_tg_trial.shape[1]

        ## according to Ohki 2020
        p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
        p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
        p_ttest = np.ones((ncell, nstim)) * np.nan
        evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

        for icell in tqdm(np.arange(ncell)):
            base_cell_anova = np.concatenate([dfof_base2_trial[icell, stim] for stim in range(nstim)])
            stim_cell_anova = [] 
            for istim in np.arange(nstim):
                stim_cell_anova.append(np.array(dfof_tg_trial[icell, istim]).flatten())

                base_cell = dfof_base2_trial[icell, istim]
                stim_cell = dfof_tg_trial[icell, istim]
                _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
                p_ttest[icell, istim] = p_ttest_i

                evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
                evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
                evoked[icell, istim] = evoked_i
            
            _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            p_anova[icell] = p_anova_cell
            _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            p_kruskal[icell] = p_kruskal_cell

    # visually driven cells: pass (anova OR kruskal) AND amp threshold for >=1 image
    p_sig = 0.05
    vis_driven = ((p_anova < p_sig) | (p_kruskal < p_sig)) & (sum(evoked.T > 0.1) > 0).reshape(-1, 1)
    print(f'{vis_driven.sum()} cells are visually driven, \n\
        proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells')

    # cells responsive to image i: pass visually driven (anova OR kruskal) AND t-test AND amp threshold for *this* image
    img_driven = vis_driven & (p_ttest < p_sig) & (evoked > 0.1)
    print(f'{img_driven.sum()} cells are image driven - with overlap between images, \n\
        proportion {np.round(img_driven.sum() / (ncell*nstim), 2)} out of {ncell*nstim} cell-stim combos. \n\
        1-N image evokes resp from {np.sum(img_driven, axis=0)} cells')

    t = np.sum(img_driven, axis=1)
    print(f'img driven cells are driven by {t[t>0]} images')

    ## save
    vis_driven_re = {}
    vis_driven_re['p_anova'] = p_anova
    vis_driven_re['p_kruskal'] = p_kruskal
    vis_driven_re['p_ttest'] = p_ttest
    vis_driven_re['evoked'] = evoked
    vis_driven_re['p_sig'] = p_sig
    vis_driven_re['vis_driven'] = vis_driven
    vis_driven_re['img_driven'] = img_driven

    os.chdir(os.path.join(dir_inter, dir_sub))
    print(os.getcwd())
    with open('vis_driven.pickle', 'wb') as f:
        pickle.dump(vis_driven_re, f)
    # break

set  4 of 36
1322 200804 003
waiting for tif


In [37]:
# # meta = pd.read_excel(mat_inter_path + 'adp_dataset_master.xlsx')

# # date_str = str(200721)
# # mouse_str = meta.loc[meta['date'] == int(date_str), 'mouse'].values#[0]
# # area_str = meta.loc[meta['date'] == int(date_str), 'area'].values[0]
# # if len(mouse_str) > 1:
# #     print('duplicate dates with maybe different mouse. select which mouse?')
# # else:
# #     mouse_str = str(mouse_str[0])
# # print(mouse_str, date_str)

# try:
#     dir_sub = area_str + '_i' + mouse_str + '_' + date_str + '_cellpose'
#     dfof_trialwise = scipy.io.loadmat(os.path.join(mat_inter_path, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
# except:
#     dir_sub = area_str + '_i' + mouse_str + '_' + date_str + ''
#     dfof_trialwise = scipy.io.loadmat(os.path.join(mat_inter_path, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
# if len(dfof_trialwise['dfof_ad_trial'].shape) == 2: # if contains only one ISI, likely 250
#     mode = 'only one isi'
#     print(f'mode is {mode} (only one isi, grat_SF or bunny)')
#     print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0].shape)
#     dfof_ad_trial = dfof_trialwise['dfof_ad_trial'] # do not subtract baseline here: stim resp will be compared to base resp
#     dfof_base_trial = dfof_trialwise['dfof_base_trial']

# if len(dfof_trialwise['dfof_ad_trial'].shape) == 3: # if contains multiple ISI, likely inf-750-250
#     mode = 'grat_8ori_3isi'
#     print(f'mode is {mode} (grating with 8 orientations and 3 ISI)')
#     print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0,0].shape)
#     dfof_tg_trial = dfof_trialwise['dfof_tg_trial'][:,:,0] # only use the first ISI, aka no adapter trials
#     dfof_base2_trial = dfof_trialwise['dfof_base2_trial'][:,:,0]
#     print('to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. \n 8 possible target orientations will cover all needed cells')

1323 200721
mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(39, 8, 3) (17, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells


In [38]:
# if mode == 'only one isi':
#     print(mode)

#     ncell = dfof_ad_trial.shape[0]
#     nstim = dfof_ad_trial.shape[1]

#     ## according to Ohki 2020
#     p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#     p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#     p_ttest = np.ones((ncell, nstim)) * np.nan
#     evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#     for icell in tqdm(np.arange(ncell)):
#         base_cell_anova = np.concatenate([dfof_base_trial[icell, stim] for stim in range(nstim)])
#         stim_cell_anova = [] 
#         for istim in np.arange(nstim):
#             stim_cell_anova.append(np.array(dfof_ad_trial[icell, istim]).flatten())

#             base_cell = dfof_base_trial[icell, istim]
#             stim_cell = dfof_ad_trial[icell, istim]
#             _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#             p_ttest[icell, istim] = p_ttest_i

#             evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#             evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#             evoked[icell, istim] = evoked_i
        
#         _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_anova[icell] = p_anova_cell
#         _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_kruskal[icell] = p_kruskal_cell

# elif mode == 'grat_8ori_3isi':
#     print(mode)

#     ncell = dfof_tg_trial.shape[0]
#     nstim = dfof_tg_trial.shape[1]

#     ## according to Ohki 2020
#     p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#     p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#     p_ttest = np.ones((ncell, nstim)) * np.nan
#     evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#     for icell in tqdm(np.arange(ncell)):
#         base_cell_anova = np.concatenate([dfof_base2_trial[icell, stim] for stim in range(nstim)])
#         stim_cell_anova = [] 
#         for istim in np.arange(nstim):
#             stim_cell_anova.append(np.array(dfof_tg_trial[icell, istim]).flatten())

#             base_cell = dfof_base2_trial[icell, istim]
#             stim_cell = dfof_tg_trial[icell, istim]
#             _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#             p_ttest[icell, istim] = p_ttest_i

#             evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#             evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#             evoked[icell, istim] = evoked_i
        
#         _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_anova[icell] = p_anova_cell
#         _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_kruskal[icell] = p_kruskal_cell

grat_8ori_3isi


100%|██████████| 39/39 [00:00<00:00, 345.57it/s]


In [45]:
# # visually driven cells: pass (anova OR kruskal) AND amp threshold for >=1 image
# p_sig = 0.05
# vis_driven = ((p_anova < p_sig) | (p_kruskal < p_sig)) & (sum(evoked.T > 0.1) > 0).reshape(-1, 1)
# print(f'{vis_driven.sum()} cells are visually driven, \n\
#     proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells')

# # cells responsive to image i: pass visually driven (anova OR kruskal) AND t-test AND amp threshold for *this* image
# img_driven = vis_driven & (p_ttest < p_sig) & (evoked > 0.1)
# print(f'{img_driven.sum()} cells are image driven - with overlap between images, \n\
#     proportion {np.round(img_driven.sum() / (ncell*nstim), 2)} out of {ncell*nstim} cell-stim combos. \n\
#     1-N image evokes resp from {np.sum(img_driven, axis=0)} cells')

# t = np.sum(img_driven, axis=1)
# print(f'img driven cells are driven by {t[t>0]} images')

26 cells are visually driven, 
    proportion 0.67 out of 39 cells
29 cells are image driven - with overlap between images, 
    proportion 0.09 out of 312 cell-stim combos. 
    1-N image evokes resp from [4 2 6 1 3 2 5 6] cells
img driven cells are driven by [1 2 2 1 2 1 1 1 2 1 2 1 4 1 1 1 1 3 1] images


In [46]:
# vis_driven_re = {}
# vis_driven_re['p_anova'] = p_anova
# vis_driven_re['p_kruskal'] = p_kruskal
# vis_driven_re['p_ttest'] = p_ttest
# vis_driven_re['evoked'] = evoked
# vis_driven_re['p_sig'] = p_sig
# vis_driven_re['vis_driven'] = vis_driven
# vis_driven_re['img_driven'] = img_driven

# os.chdir(os.path.join(mat_inter_path, dir_sub))
# print(os.getcwd())

# with open('vis_driven.pickle', 'wb') as f:
#     pickle.dump(vis_driven_re, f)

\\duhs-user-nc1.dhe.duke.edu\dusom_glickfeldlab\All_Staff\home\lan\Data\2P_images\mat_inter\LM_i1323_200721
